In [ ]:
from nos.client import InferenceClient, TaskType

### Inference Client API

In [ ]:
client = InferenceClient()

In [ ]:
client.IsHealthy()

In [ ]:
client.WaitForServer()

### List all available models

In [ ]:
models = client.ListModels()
models

### Get model input/output signatures

In [ ]:
for spec in models:
    sig = client.GetModelInfo(spec).signature
    print("-" * 80)
    print(f"task={spec.task}, name={spec.name}")
    
    inputs, outputs = sig.get_inputs_spec(), sig.get_outputs_spec()
    print(f"inputs: {inputs}")
    print(f"outputs: {outputs}")

### Load image

In [ ]:
import requests
from PIL import Image

url = "https://raw.githubusercontent.com/open-mmlab/mmdetection/main/demo/demo.jpg"
img = Image.open(requests.get(url, stream=True).raw)

### Run text-to-image generation with StableDiffusionV2

In [ ]:
prompts = ["fox jumped over the moon", "fox jumped over the sun"]
predictions = client.Run(TaskType.IMAGE_GENERATION, "stabilityai/stable-diffusion-2", 
                         prompts=prompts, width=512, height=512, num_images=1)
for prompt, image in zip(prompts, predictions["images"]):
    print(prompt, image.size)

### Run image-embedding with CLIP

In [ ]:
predictions = client.Run(TaskType.IMAGE_EMBEDDING, "openai/clip", images=[img, img])
predictions["embedding"].shape

### Run text-embedding with CLIP

In [ ]:
predictions = client.Run(TaskType.TEXT_EMBEDDING, "openai/clip", texts=["fox jumped over the mooon", "fox jumped over the sun"])
predictions["embedding"].shape

### Run object detection with FasterRCNN

In [ ]:
predictions = client.Run(TaskType.OBJECT_DETECTION_2D, "torchvision/fasterrcnn_mobilenet_v3_large_320_fpn", images=[img])
for idx, (bboxes, scores, labels) in enumerate(zip(predictions["bboxes"], predictions["scores"], predictions["scores"])):
    print(idx, bboxes.shape, scores.shape, labels.shape)

### `client.Module` Interface

Get model handles for simpler remote-model execution. In this case, the following line re-uses the SDv2 model instantiated earlier.  

In [ ]:
sdv2 = client.Module(TaskType.IMAGE_GENERATION, "stabilityai/stable-diffusion-2")
predictions = sdv2(prompts=["fox in the rain at night, 4k, hdr"], width=512, height=512, num_images=1)
predictions["images"]

In [ ]:
fastrcnn = client.Module(TaskType.OBJECT_DETECTION_2D, "torchvision/fasterrcnn_mobilenet_v3_large_320_fpn")
predictions = fastrcnn(images=[img])
for idx, (bboxes, scores, labels) in enumerate(zip(predictions["bboxes"], predictions["scores"], predictions["scores"])):
    print(idx, bboxes.shape, scores.shape, labels.shape)